In [1]:
from datetime import datetime, timezone
import serial
import struct
import numpy as np
from typing import NamedTuple

In [2]:
PORT = 'COM255'
BAUDRATE = 115200
TIMEOUT = 1

In [3]:
class FLAGS:
    """QARTOD-ESQUE FLAGS"""
    PASS: int = 1
    NOT_EVALUATED: int = 2
    SUSPECT: int = 3
    FAIL: int = 4
    MISSING_DATA: int = 9

@staticmethod
def syntax_test(packet, record_length):
    """
    This syntax test is specific to the ACS and its quirks. It should satisfy QARTOD requirements based on the description for the syntax test of other parameters.
    
    :param packet: The full packet from the ACS, including registration bytes, the checksum, and pad byte.
    :param record_length: The record length reported by the ACS.
    :return: FAIL if the packet length and checksum does-not-equal checks are True. PASS if the does-not-equal checks are False.
    """
    PAD_BYTE = b'\x00'
    pad_byte_idx = packet.rfind(PAD_BYTE) # Pad byte index should always represent the last byte in the packet.
    checksum = packet[pad_byte_idx-2:pad_byte_idx]  # The checksum is the two bytes that occur before the pad 
    if len(packet) != record_length + 3:  # The record_length output with each packet does not account for the checksum and pad byte, so add 3 bytes to the record_length.
        return FLAGS.FAIL
    elif np.array(sum(packet[:-3])).astype(np.uint16) != struct.unpack_from('!H', checksum): # Taken from Inlinino. Verifies the checksum.
        return FLAGS.FAIL
    else:
        return FLAGS.PASS
    
class RawPacket(NamedTuple):
    """Container for a raw packet."""
    record_length: int
    packet_type: int
    reserved_1: int
    serial_number_hexdec: int
    a_reference_dark: int
    pressure_counts: int
    a_signal_dark: int
    raw_external_temp: int
    raw_internal_temp: int
    c_reference_dark: int
    c_signal_dark: int
    elapsed_time: int
    reserved_2: int
    number_of_output_wavelengths: int
    c_reference: list
    a_reference: list
    c_signal: list
    a_signal: list    

In [4]:
PACKET_REGISTRATION = b'\xff\x00\xff\x00'
PAD_BYTE = b'\x00'
PACKET_HEAD = '!4cHBBl7HIBB'
PACKET_TAIL = 'Hx'
LEN_PREDEFINED = struct.calcsize(PACKET_HEAD + PACKET_TAIL)

In [5]:
sample_num = 0
buffer = bytearray()
sample_dict = {}
with serial.Serial(port = PORT, baudrate = BAUDRATE, timeout = TIMEOUT) as rs:
    rs.reset_output_buffer()
    rs.reset_input_buffer()
    while True:
        if sample_num == 10:  # If we have taken 10 samples, exit the while loop.
            break
        buffer.extend(rs.read(rs.in_waiting))
        old_bytes, sep, remaining_bytes = buffer.partition(PACKET_REGISTRATION)
        if PACKET_REGISTRATION in remaining_bytes:
            dt = datetime.now(timezone.utc)
            packet_data, next_sep, next_loop_bytes = remaining_bytes.partition(PACKET_REGISTRATION)
            packet = sep + packet_data
            buffer = next_sep + next_loop_bytes # Redefine buffer.
            pad_byte_idx = packet.rfind(PAD_BYTE) # Pad byte index should always represent the last byte in the packet.
            checksum = packet[pad_byte_idx-2:pad_byte_idx]  # The checksum is the two bytes that occur before the pad 

            # Option 1: Use the size of the PACKET HEAD and PACKET TAIL removed from the total packet length to get the descriptor.
            size_of_remaining_bytes = int((len(packet) - LEN_PREDEFINED)/2)
            descriptor1 = PACKET_HEAD + f"{size_of_remaining_bytes}H" + PACKET_TAIL
            raw1 = struct.unpack_from(descriptor1, packet)
            raw1 = raw1[4:] # Drop the packet registration

            #Option 2: Parse out the number of wavelengths from the packet first and use that to build the descriptor. nwvls * 4
            nwvls = packet[31]
            size_of_remaining_bytes = int(nwvls * 2 * 2)
            descriptor2 = PACKET_HEAD + f"{size_of_remaining_bytes}H" + PACKET_TAIL
            raw2 = struct.unpack_from(descriptor2, packet)
            raw2 = raw2[4:] # Drop the packet registration
  
            raw_packet_1 = RawPacket(record_length=raw1[0],
                                   packet_type = raw1[1],
                                   reserved_1 = raw1[2],
                                   serial_number_hexdec=raw1[3], # This is a combination of the sensor-type and serial number. Defined in the struct unpack as 'l'.
                                   a_reference_dark=raw1[4],
                                   pressure_counts=raw1[5], # Pressure counts can be ignored. See ACS Manual (Rev N).
                                   a_signal_dark= raw1[6],
                                   raw_external_temp=raw1[7],
                                   raw_internal_temp=raw1[8],
                                   c_reference_dark=raw1[9],
                                   c_signal_dark=raw1[10],
                                   elapsed_time=raw1[11],
                                   reserved_2=raw1[12],
                                   number_of_output_wavelengths=raw1[13],
                                   c_reference = raw1[14::4],
                                   a_reference = raw1[15::4],
                                   c_signal = raw1[16::4],
                                   a_signal = raw1[17::4])

            raw_packet_2 = RawPacket(record_length=raw2[0],
                                   packet_type = raw2[1],
                                   reserved_1 = raw2[2],
                                   serial_number_hexdec=raw2[3], # This is a combination of the sensor-type and serial number. Defined in the struct unpack as 'l'.
                                   a_reference_dark=raw2[4],
                                   pressure_counts=raw2[5], # Pressure counts can be ignored. See ACS Manual (Rev N).
                                   a_signal_dark= raw2[6],
                                   raw_external_temp=raw2[7],
                                   raw_internal_temp=raw2[8],
                                   c_reference_dark=raw2[9],
                                   c_signal_dark=raw2[10],
                                   elapsed_time=raw2[11],
                                   reserved_2=raw2[12],
                                   number_of_output_wavelengths=raw2[13],
                                   c_reference = raw2[14::4],
                                   a_reference = raw2[15::4],
                                   c_signal = raw2[16::4],
                                   a_signal = raw2[17::4])

            sample_dict[sample_num] = {'time': dt, 'raw_packet_1': raw_packet_1,'raw_packet_2': raw_packet_2, 'syntax_test_1': syntax_test(packet, raw_packet_1.record_length), 'syntax_test_2': syntax_test(packet, raw_packet_2.record_length)}
            sample_num +=1

In [6]:
print('Packet Descriptor From Option 1:', descriptor1)
print('Packet Descriptor From Option 2:', descriptor2)
print('descriptor1 == descriptor2:', descriptor1 == descriptor2)

Packet Descriptor From Option 1: !4cHBBl7HIBB336HHx
Packet Descriptor From Option 2: !4cHBBl7HIBB336HHx
descriptor1 == descriptor2: True


In [7]:
for sample, details in sample_dict.items():
    print(details['time'])

2024-02-08 05:51:11.990596+00:00
2024-02-08 05:51:12.240844+00:00
2024-02-08 05:51:12.490200+00:00
2024-02-08 05:51:12.740891+00:00
2024-02-08 05:51:12.990558+00:00
2024-02-08 05:51:13.240186+00:00
2024-02-08 05:51:13.490207+00:00
2024-02-08 05:51:13.739728+00:00
2024-02-08 05:51:13.989744+00:00
2024-02-08 05:51:14.239766+00:00


In [8]:
print(sample_dict[9]['raw_packet_2'])

RawPacket(record_length=704, packet_type=5, reserved_1=1, serial_number_hexdec=1392508939, a_reference_dark=464, pressure_counts=0, a_signal_dark=2116, raw_external_temp=30294, raw_internal_temp=44753, c_reference_dark=480, c_signal_dark=715, elapsed_time=7522345, reserved_2=1, number_of_output_wavelengths=84, c_reference=(524, 600, 681, 779, 890, 1013, 1139, 1286, 1449, 1636, 1841, 2047, 2253, 2480, 2727, 2987, 3263, 3569, 3886, 4249, 4626, 5012, 5417, 5849, 6270, 6696, 7104, 7517, 7984, 8446, 8888, 9371, 9892, 10477, 11130, 11743, 12376, 12955, 13541, 14154, 14661, 15183, 15707, 16195, 16770, 17294, 18724, 19169, 19600, 19935, 20279, 20585, 20870, 21132, 21233, 21247, 21226, 21145, 21004, 20826, 20512, 20037, 19485, 18811, 18474, 18306, 17757, 17148, 16534, 15821, 15140, 14506, 13795, 12986, 12176, 11502, 10886, 10209, 9525, 8896, 8268, 7694, 7168, 6635, 63217), a_reference=(405, 471, 545, 631, 735, 843, 961, 1102, 1256, 1434, 1626, 1814, 2017, 2235, 2478, 2729, 3004, 3295, 3612, 396